__`NOTEBOOK START`__

-----------

#### __NOTEBOOK PLAN__ 🎯

1. Get the gps coordinates of all the cities from nominatim
2. Get weather data from each destination
3. Get hotels' info about each destination
4. Store all the information above in a data lake
5. Extract, transform and load cleaned data from your datalake to a data warehouse

-----------

In [43]:
print('>>>> 📚 Importing librairies...')
import requests

import pandas as pd

import json

from time import sleep

import plotly
import plotly.express as px

from datetime import datetime, timezone

import scrapy
from scrapy.crawler import CrawlerProcess

import logging

import datetime

import os

import boto3

import bs4

from sqlalchemy import create_engine

print('>>>> ✅ ...Done')

>>>> 📚 Importing librairies...
>>>> ✅ ...Done


In [44]:
# Scrape data from these destinations
destinations_to_scrape = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",\
"Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon",\
"Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer",\
"Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"] 

# we order them in alphabetic order
destinations_to_scrape.sort()

print(destinations_to_scrape)

['Aigues Mortes', 'Aix en Provence', 'Amiens', 'Annecy', 'Ariege', 'Avignon', 'Bayeux', 'Bayonne', 'Besancon', 'Biarritz', 'Bormes les Mimosas', 'Carcassonne', 'Cassis', 'Chateau du Haut Koenigsbourg', 'Collioure', 'Colmar', 'Dijon', 'Eguisheim', 'Gorges du Verdon', 'Grenoble', 'La Rochelle', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Mont Saint Michel', 'Montauban', 'Nimes', 'Paris', 'Rouen', 'Saintes Maries de la mer', 'St Malo', 'Strasbourg', 'Toulouse', 'Uzes']


In [45]:
print ('Number of cities to scrap information for:', len(destinations_to_scrape))

Number of cities to scrap information for: 35


---------
### __1. Get the gps coordinates of all the cities from nominatim__
---------

In [46]:
url_nominatim = 'https://nominatim.openstreetmap.org/search?'

def get_gps_coordinates (address, format='json', limit=1, country='France'):
    # we create a dictionary with the values of the arguments that will be passed as parameters to the API request
    params = {'q': address, 'format': format, 'limit': limit, 'country': country}
    # then we send a get request to the API with the URL and parameters
    r = requests.get(url_nominatim, params=params)
    # we try avoiding overloading with the sleep function
    sleep(1)
    return r.json()

In [47]:
# these will be the columns names of our cities_df
columns = ['cid', 'city', 'latitude', 'longitude']

# we create an empyt list in which will store the information we get
data = []

# now we loop over each city in our list of cities
for count, city in enumerate(destinations_to_scrape):
    # for each city, we call the function with the address argument set to the name of the city
    response = get_gps_coordinates(address=city)
    # the 'row' list will contain the data of the current city and the count variable
    # assigns a unique ID to each city stored in the column cid
    row = [count, city, response[0]['lat'], response[0]['lon']]
    # we then append this list to the 'data' list
    data.append(row)

# we now convert our list to a Data frame
cities_df = pd.DataFrame(data=data, columns=columns).sort_values(by='city', ascending=True).reset_index(drop='index')

In [48]:
print()
print('Display of dataset: ')
print()
display(cities_df.head(11))

print()
print('Main statistics: ')
print()
df_stats = cities_df.describe().apply(lambda s: s.apply('{0:.2f}'.format))
display(df_stats)

print()
print ('Missing values:')
print()
# below is a function that will return the percentage and the quantity of missing values in our DF
def missing_values():
    
    missing = (pd.DataFrame(cities_df.isnull().sum()/cities_df.shape[0])*100).round(2)
    missing.columns = ['Percentage of Missing Values']
    missing['Number of Missing Values'] = pd.DataFrame(cities_df.isnull().sum())
    
    return missing.sort_values(by='Percentage of Missing Values', ascending=False)

display(missing_values())


Display of dataset: 



,cid,city,latitude,longitude
0,0,Aigues Mortes,43.5658225,4.1912837
1,1,Aix en Provence,43.5298424,5.4474738
2,2,Amiens,49.8941708,2.2956951
3,3,Annecy,45.8992348,6.1288847
4,4,Ariege,42.9455368,1.4065544156065486
5,5,Avignon,43.9492493,4.8059012
6,6,Bayeux,49.2764624,-0.7024738
7,7,Bayonne,43.4945144,-1.4736657
8,8,Besancon,47.2380222,6.0243622
9,9,Biarritz,43.471143749999996,-1.552726590666314



Main statistics: 



,cid
count,35.00
mean,17.00
std,10.25
min,0.00
25%,8.50
50%,17.00
75%,25.50
max,34.00



Missing values:



,Percentage of Missing Values,Number of Missing Values
cid,0.0,0
city,0.0,0
latitude,0.0,0
longitude,0.0,0


In [49]:
cities_df.dtypes

cid           int64
city         object
latitude     object
longitude    object
dtype: object

In [50]:
# we convert our lat and long data to float
cities_df[['latitude', 'longitude']] = cities_df[['latitude', 'longitude']].astype(float)

In [51]:
cities_df.dtypes

cid            int64
city          object
latitude     float64
longitude    float64
dtype: object

### Let's visualize our cities on a map

In [52]:
fig = px.scatter_mapbox(
        cities_df, 
        lat="latitude", 
        lon="longitude",
        color="city",
        mapbox_style="carto-positron",
        zoom=3.7,
        title='Top 35 Cities Location'
)

fig.show()

In [53]:
"""# we save our information
cities_df.to_csv('cities_lat_lon.csv', index = False)"""

"# we save our information\ncities_df.to_csv('cities_lat_lon.csv', index = False)"

---------
### __2. Get weather data from open_weather API__
---------

In [54]:
key = pd.read_csv('OpenWeather_API_KEY.txt')

In [55]:
url_open_weather_map = 'https://api.openweathermap.org/data/3.0/onecall?'

def get_weather_info (lat, lon, exclude, API_key = key, units= 'metric'):
    # again, with this function we create a dictionary with the parameters that will be passed to the API request
    params = {'lat': lat, 'lon': lon, 'exclude': exclude, 'appid': API_key, 'units': units}
    r = requests.get(url_open_weather_map, params=params)
    sleep(1)
    return r.json()

In [56]:
def convert_day_time(unixDt):
    utc_time = datetime.datetime.fromtimestamp(unixDt, timezone.utc)
    local_time = utc_time.astimezone()
    return (local_time.strftime("%Y-%m-%d %H:%M:%S"))

In [57]:
columns = ['day_time', 'temperature', 'probability_to_rain', 'weather_description', 'cid']
weather_desc = []

# we loop over every cities in our cities_df to extract the cid, the lat and lon
for i in range(len(cities_df)):
    
    cid = cities_df.loc[i, 'cid']
    latitude = cities_df.loc[i, 'latitude']
    longitude = cities_df.loc[i, 'longitude']

    # we call our function to obtain the weather information and loop to get the next 7 days
    r_weather = get_weather_info(lat=latitude, lon=longitude, exclude='current, minutely, hourly, alerts')
    
    for j in range(1, 8):

        day_time = convert_day_time(int(r_weather['daily'][j]['dt']))
        temperature = r_weather['daily'][j]['temp']['day']
        probability_to_rain = r_weather['daily'][j]['pop']
        weather = r_weather['daily'][j]['weather'][0]['description']
        
        weather_desc.append([day_time, temperature, probability_to_rain, weather, cid])
    
weather_df = pd.DataFrame(weather_desc, columns=columns)

In [58]:
print()
print('Display of dataset: ')
print()
display(weather_df.head(11))

print()
print('Main statistics: ')
print()
df_stats = weather_df.describe().apply(lambda s: s.apply('{0:.2f}'.format))
display(df_stats)

print()
print ('Missing values:')
print()
# Here I created a function that will return the percentage and the quantity of missing values in our DF
def missing_values():
    
    missing = (pd.DataFrame(weather_df.isnull().sum()/weather_df.shape[0])*100).round(2)
    missing.columns = ['Percentage of Missing Values']
    missing['Number of Missing Values'] = pd.DataFrame(weather_df.isnull().sum())
    
    return missing.sort_values(by='Percentage of Missing Values', ascending=False)

display(missing_values())


Display of dataset: 



,day_time,temperature,probability_to_rain,weather_description,cid
0,2023-05-10 13:00:00,20.14,0.00,clear sky,0
1,2023-05-11 13:00:00,19.37,0.00,scattered clouds,0
2,2023-05-12 13:00:00,17.43,0.30,light rain,0
3,2023-05-13 13:00:00,18.31,0.73,light rain,0
4,2023-05-14 13:00:00,22.53,0.39,scattered clouds,0
5,2023-05-15 13:00:00,17.18,0.00,overcast clouds,0
6,2023-05-16 13:00:00,17.59,0.09,few clouds,0
7,2023-05-10 13:00:00,18.76,0.37,clear sky,1
8,2023-05-11 13:00:00,18.14,0.00,clear sky,1
9,2023-05-12 13:00:00,17.66,0.63,light rain,1



Main statistics: 



,temperature,probability_to_rain,cid
count,245.00,245.00,245.00
mean,14.92,0.66,17.00
std,3.37,0.36,10.12
min,5.39,0.00,0.00
25%,12.53,0.39,8.00
50%,15.01,0.84,17.00
75%,17.59,1.00,26.00
max,22.58,1.00,34.00



Missing values:



,Percentage of Missing Values,Number of Missing Values
day_time,0.0,0
temperature,0.0,0
probability_to_rain,0.0,0
weather_description,0.0,0
cid,0.0,0


In [59]:
len(weather_df)

245

In [60]:
"""# we save our information
weather_df.to_csv('weather_forecast.csv', index = False)"""

"# we save our information\nweather_df.to_csv('weather_forecast.csv', index = False)"

---------
### __3. Scrap Booking.com__
---------

To take a look at our spider, please open the file named: 'Scrap_Booking.py'

In [61]:
# the information scraped by our spider has been stored in the file called:'booking_hotel_scraping.json'
booking_df = pd.read_json('booking_hotel_scraping_1.json')

In [62]:
print()
print('Display of dataset: ')
print()
display(booking_df.head(11))

print()
print('Main statistics: ')
print()
df_stats = booking_df.describe()
display(df_stats)

print()
print ('Missing values:')
print()
# Here I created a function that will return the percentage and the quantity of missing values in our DF
def missing_values():
    
    missing = (pd.DataFrame(booking_df.isnull().sum()/booking_df.shape[0])*100).round(2)
    missing.columns = ['Percentage of Missing Values']
    missing['Number of Missing Values'] = pd.DataFrame(booking_df.isnull().sum())
    
    return missing.sort_values(by='Percentage of Missing Values', ascending=False)

display(missing_values())


Display of dataset: 



,city,name,description,rating,location,url
0,Amiens,WHITE HOUSE DHAVERNAS - PROCHE CENTRE - PARKIN...,"Situé à Amiens, en Picardie, à proximité de la...","8,7",Amiens,https://www.booking.com/hotel/fr/white-house-d...
1,Amiens,La Parenthèse Envoutée,"Situé à Amiens, en Picardie, à proximité de la...","9,0",Amiens,https://www.booking.com/hotel/fr/aux-caves.fr....
2,Amiens,"Holiday Inn Express Amiens, an IHG Hotel","Rénové en avril 2015, le Holiday Inn Express A...","7,9",Amiens,https://www.booking.com/hotel/fr/express-by-ho...
3,Amiens,L'Oratoire,"Situé à Amiens, à proximité de la gare, du par...","8,0",Amiens,https://www.booking.com/hotel/fr/oratoire-amie...
4,Amiens,Hotel Le Prieuré et La Résidence,L'Hotel Le Prieuré et La Résidence est situé d...,"8,3",Amiens,https://www.booking.com/hotel/fr/le-prieure-am...
5,Amiens,Le Cottage des Hortillonnages,Le Cottage des Hortillonnages est situé à Amie...,"9,3",Amiens,https://www.booking.com/hotel/fr/le-cottage-de...
6,Amiens,Les Augustins,"Situé à Amiens, à moins de 700 mètres de la ga...","7,9",Amiens,https://www.booking.com/hotel/fr/les-augustins...
7,Amiens,Moxy Amiens,Situé au cœur de la ville et à deux pas de la ...,"8,5",Amiens,https://www.booking.com/hotel/fr/campanile-ami...
8,Amiens,Chill & Sun,Doté d'une terrasse et offrant une vue sur le ...,"8,7",Amiens,https://www.booking.com/hotel/fr/t2-industriel...
9,Amiens,Odalys City Amiens Blamont,"Situé à 2,8 km du Zénith d'Amiens et à 900 mèt...","8,0",Amiens,https://www.booking.com/hotel/fr/appart-39-oda...



Main statistics: 



,city,name,description,rating,location,url
count,875,875,875,847,875,875
unique,35,872,875,37,136,875
top,Amiens,Hôtel Le Médiéval,"Situé à Amiens, en Picardie, à proximité de la...","8,0",Amiens,https://www.booking.com/hotel/fr/white-house-d...
freq,25,2,1,59,25,1



Missing values:



,Percentage of Missing Values,Number of Missing Values
rating,3.2,28
city,0.0,0
name,0.0,0
description,0.0,0
location,0.0,0
url,0.0,0


In [63]:
# we put 0 where values are missing in the column 'rating' 
booking_df['rating'] = booking_df['rating'].fillna(0)

In [64]:
booking_df.dtypes

city           object
name           object
description    object
rating         object
location       object
url            object
dtype: object

In [65]:
# we replace the comma by a dot in order to change the type of the column rating
booking_df['rating'] = booking_df['rating'].replace({",":"."}, regex=True)

# now we can change the type
booking_df['rating'] = booking_df['rating'].astype(float)

In [66]:
booking_df.dtypes

city            object
name            object
description     object
rating         float64
location        object
url             object
dtype: object

In [67]:
# for each city we are going to look for the corresponding id in the df cities_df
# this will help us when using SQL
booking_df['cid'] = booking_df['city'].map(cities_df.set_index('city')['cid'])

In [68]:
booking_df

,city,name,description,rating,location,url,cid
0,Amiens,WHITE HOUSE DHAVERNAS - PROCHE CENTRE - PARKIN...,"Situé à Amiens, en Picardie, à proximité de la...",8.7,Amiens,https://www.booking.com/hotel/fr/white-house-d...,2
1,Amiens,La Parenthèse Envoutée,"Situé à Amiens, en Picardie, à proximité de la...",9.0,Amiens,https://www.booking.com/hotel/fr/aux-caves.fr....,2
2,Amiens,"Holiday Inn Express Amiens, an IHG Hotel","Rénové en avril 2015, le Holiday Inn Express A...",7.9,Amiens,https://www.booking.com/hotel/fr/express-by-ho...,2
3,Amiens,L'Oratoire,"Situé à Amiens, à proximité de la gare, du par...",8.0,Amiens,https://www.booking.com/hotel/fr/oratoire-amie...,2
4,Amiens,Hotel Le Prieuré et La Résidence,L'Hotel Le Prieuré et La Résidence est situé d...,8.3,Amiens,https://www.booking.com/hotel/fr/le-prieure-am...,2
...,...,...,...,...,...,...,...
870,Bayonne,Studio centre Bayonne,"Situé à Bayonne, en Aquitaine, le Studio centr...",8.2,Bayonne,https://www.booking.com/hotel/fr/studio-centre...,7
871,Bayonne,Hôtel des Arceaux,Situé dans une zone piétonne du centre histori...,7.0,Bayonne,https://www.booking.com/hotel/fr/des-arceaux.f...,7
872,Bayonne,Campanile Bayonne,Situé à 5 minutes en voiture du centre de Bayo...,6.6,Bayonne,https://www.booking.com/hotel/fr/campanile-bay...,7
873,Bayonne,La Chambre d'Hote de Mano - Centre-ville de Ba...,"Situé à 200 mètres de la gare de Bayonne, à mo...",9.4,Bayonne,https://www.booking.com/hotel/fr/chambre-d-39-...,7


In [69]:
# checking how many hotels we have, as we should have the number of hotels displayed on 1 page, which is 25
num_of_hotels = booking_df[booking_df['city']=='Aix en Provence']['name'].value_counts().sum()
num_of_hotels

25

In [70]:
booking_df[booking_df['city']=='Aix en Provence']['rating']

450    8.0
451    7.3
452    0.0
453    8.3
454    8.0
455    8.3
456    7.8
457    8.1
458    7.9
459    8.1
460    7.9
461    8.4
462    8.0
465    7.8
466    7.9
467    7.7
468    7.1
469    9.0
470    8.1
471    7.7
472    8.4
473    8.3
474    7.6
475    7.2
476    7.4
Name: rating, dtype: float64

In [71]:
"""# we save our information
booking_df.to_csv('booking_scraping_df.csv', index = False)"""

"# we save our information\nbooking_df.to_csv('booking_scraping_df.csv', index = False)"

We now add the information from the lat and lon from our second spider.

In [72]:
# like above, we convert it to a pd dataframe
df_booking_latlon = pd.read_json ('booking_hotel_scraping_2.json')
df_booking_latlon = pd.DataFrame(df_booking_latlon)
df_booking_latlon

,lat_lon
0,"49.89755206,2.31635948"
1,"49.89444519,2.30094239"
2,"49.88146960,2.29987140"
3,"49.88833910,2.31217084"
4,"49.89422900,2.30577800"
...,...
870,"43.49224970,-1.47577260"
871,"43.49530232,-1.46939464"
872,"43.48954193,-1.47812448"
873,"43.49112493,-1.45484090"


In [73]:
# we also need to split the lat and lon that comes in one part
df_booking_latlon = df_booking_latlon['lat_lon'].str.split(',', expand=True)
df_booking_latlon['lat_H'] = df_booking_latlon[0]
df_booking_latlon['lon_H'] = df_booking_latlon[1]
df_booking_latlon = df_booking_latlon[['lat_H', 'lon_H']]
df_booking_latlon

,lat_H,lon_H
0,49.89755206,2.31635948
1,49.89444519,2.30094239
2,49.88146960,2.29987140
3,49.88833910,2.31217084
4,49.89422900,2.30577800
...,...,...
870,43.49224970,-1.47577260
871,43.49530232,-1.46939464
872,43.48954193,-1.47812448
873,43.49112493,-1.45484090


In [74]:
# we convert the columns into float
df_booking_latlon[['lat_H', 'lon_H']] = df_booking_latlon[['lat_H', 'lon_H']] .astype(float)

In [75]:
# we join all the informations
booking_df_with_latlon = booking_df.join(df_booking_latlon)
booking_df_with_latlon

,city,name,description,rating,location,url,cid,lat_H,lon_H
0,Amiens,WHITE HOUSE DHAVERNAS - PROCHE CENTRE - PARKIN...,"Situé à Amiens, en Picardie, à proximité de la...",8.7,Amiens,https://www.booking.com/hotel/fr/white-house-d...,2,49.897552,2.316359
1,Amiens,La Parenthèse Envoutée,"Situé à Amiens, en Picardie, à proximité de la...",9.0,Amiens,https://www.booking.com/hotel/fr/aux-caves.fr....,2,49.894445,2.300942
2,Amiens,"Holiday Inn Express Amiens, an IHG Hotel","Rénové en avril 2015, le Holiday Inn Express A...",7.9,Amiens,https://www.booking.com/hotel/fr/express-by-ho...,2,49.881470,2.299871
3,Amiens,L'Oratoire,"Situé à Amiens, à proximité de la gare, du par...",8.0,Amiens,https://www.booking.com/hotel/fr/oratoire-amie...,2,49.888339,2.312171
4,Amiens,Hotel Le Prieuré et La Résidence,L'Hotel Le Prieuré et La Résidence est situé d...,8.3,Amiens,https://www.booking.com/hotel/fr/le-prieure-am...,2,49.894229,2.305778
...,...,...,...,...,...,...,...,...,...
870,Bayonne,Studio centre Bayonne,"Situé à Bayonne, en Aquitaine, le Studio centr...",8.2,Bayonne,https://www.booking.com/hotel/fr/studio-centre...,7,43.492250,-1.475773
871,Bayonne,Hôtel des Arceaux,Situé dans une zone piétonne du centre histori...,7.0,Bayonne,https://www.booking.com/hotel/fr/des-arceaux.f...,7,43.495302,-1.469395
872,Bayonne,Campanile Bayonne,Situé à 5 minutes en voiture du centre de Bayo...,6.6,Bayonne,https://www.booking.com/hotel/fr/campanile-bay...,7,43.489542,-1.478124
873,Bayonne,La Chambre d'Hote de Mano - Centre-ville de Ba...,"Situé à 200 mètres de la gare de Bayonne, à mo...",9.4,Bayonne,https://www.booking.com/hotel/fr/chambre-d-39-...,7,43.491125,-1.454841


---------
### __4. Create our data lake using AWS S3__
---------

In [76]:
"""os.environ['aws_access_key_id'] = 'XXXXX'
os.environ['aws_secret_access_key'] = 'XXXXX'"""

In [77]:
# creating our boto3 session in order to interact with AWS
session = boto3.Session(
    aws_access_key_id = os.getenv('aws_access_key_id'),
    aws_secret_access_key = os.getenv('aws_secret_access_key')
) 

In [78]:
# create an S3 resource object to interact with our futur bucket
s3 = session.resource('s3')

In [79]:
# this is the bucket that will hold all our df
bucket = s3.create_bucket(Bucket='kayak.project.booking.scraping')

# we save cities_df
csv_1 = cities_df.to_csv(index = False)

# we save weather_df
csv_2 = weather_df.to_csv(index = False)

# we save booking_df
csv_3 = booking_df.to_csv(index = False)

# we now can transfert those df to our bucket
bucket.put_object(Key='cities_lat_lon_df.csv', Body = csv_1)
bucket.put_object(Key='weather_forecast_df.csv', Body = csv_2)
bucket.put_object(Key='booking_scraping_df.csv', Body = csv_3)

s3.Object(bucket_name='kayak.project.booking.scraping', key='booking_scraping_df.csv')

---------
### __5. Extract, Transform and Load: Create our data lake using AWS RDS__
---------

We are now going to create a SQL Database using AWS RDS in order to extract our data from S3 and store it in your newly created DB

In [80]:
"""os.environ['master_username'] = 'admin'
os.environ['master_password'] = 'XXXXX'
os.environ['endpoint'] = 'XXXXX'
os.environ['port'] = 'XXXXX'"""

In [81]:
master_username = os.getenv('master_username')
master_password = os.getenv('master_password')
endpoint = os.getenv('endpoint')
port = os.getenv('port')
name = os.getenv('name')

# we call the function create_engine and set the echo to true which enables logging of the SQL statements executed by the engine
engine = create_engine(f'mysql+pymysql://{master_username}:{master_password}@{endpoint}:{port}/', echo=True)

In [82]:
#engine.execute("DROP DATABASE db1")

In [83]:
engine.execute("CREATE DATABASE db1")

2023-05-09 15:54:20,416 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-09 15:54:20,417 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-09 15:54:20,456 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-09 15:54:20,457 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-09 15:54:20,469 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-09 15:54:20,470 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-09 15:54:20,495 INFO sqlalchemy.engine.Engine CREATE DATABASE db1
2023-05-09 15:54:20,496 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-09 15:54:20,524 INFO sqlalchemy.engine.Engine COMMIT


In [84]:
#these are the data frame stored in my s3 bucket
for i in bucket.objects.all():
    print(i.key)

booking_scraping_df.csv
cities_lat_lon_df.csv
weather_forecast_df.csv


In [85]:
engine.execute('USE db1')

2023-05-09 15:54:21,073 INFO sqlalchemy.engine.Engine USE db1
2023-05-09 15:54:21,075 INFO sqlalchemy.engine.Engine [raw sql] {}


In [86]:
booking_df_with_latlon.to_sql('booking_df', engine)

2023-05-09 15:54:21,164 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:21,165 INFO sqlalchemy.engine.Engine [generated in 0.00175s] {'table_schema': 'None', 'table_name': 'booking_df'}
2023-05-09 15:54:21,187 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 15:54:21,190 INFO sqlalchemy.engine.Engine 
CREATE TABLE booking_df (
	`index` BIGINT, 
	city TEXT, 
	name TEXT, 
	description TEXT, 
	rating FLOAT(53), 
	location TEXT, 
	url TEXT, 
	cid BIGINT, 
	`lat_H` FLOAT(53), 
	`lon_H` FLOAT(53)
)


2023-05-09 15:54:21,191 INFO sqlalchemy.engine.Engine [no key 0.00151s] {}
2023-05-09 15:54:21,293 INFO sqlalchemy.engine.Engine CREATE INDEX ix_booking_df_index ON booking_df (`index`)
2023-05-09 15:54:21,294 INFO sqlalchemy.engine.Engine [no key 0.00117s] {}
2023-05-09 15:54:21,353 INFO sqlalchemy.engine.Engine COMMIT
2023-05-09 15:54:21,372 INFO sqlalchemy.engine.Engine

875

In [87]:
cities_df.to_sql('cities_df', engine, if_exists='replace')

2023-05-09 15:54:22,098 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:22,099 INFO sqlalchemy.engine.Engine [cached since 0.9357s ago] {'table_schema': 'None', 'table_name': 'cities_df'}
2023-05-09 15:54:22,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 15:54:22,144 INFO sqlalchemy.engine.Engine 
CREATE TABLE cities_df (
	`index` BIGINT, 
	cid BIGINT, 
	city TEXT, 
	latitude FLOAT(53), 
	longitude FLOAT(53)
)


2023-05-09 15:54:22,145 INFO sqlalchemy.engine.Engine [no key 0.00086s] {}
2023-05-09 15:54:22,191 INFO sqlalchemy.engine.Engine CREATE INDEX ix_cities_df_index ON cities_df (`index`)
2023-05-09 15:54:22,191 INFO sqlalchemy.engine.Engine [no key 0.00089s] {}
2023-05-09 15:54:22,228 INFO sqlalchemy.engine.Engine COMMIT
2023-05-09 15:54:22,241 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 15:54:22,244 INFO sqlalchemy.engine.Engine INSERT I

35

In [88]:
weather_df.to_sql('weather_df', engine, if_exists='replace')

2023-05-09 15:54:22,385 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:22,386 INFO sqlalchemy.engine.Engine [cached since 1.223s ago] {'table_schema': 'None', 'table_name': 'weather_df'}
2023-05-09 15:54:22,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 15:54:22,415 INFO sqlalchemy.engine.Engine 
CREATE TABLE weather_df (
	`index` BIGINT, 
	day_time TEXT, 
	temperature FLOAT(53), 
	probability_to_rain FLOAT(53), 
	weather_description TEXT, 
	cid BIGINT
)


2023-05-09 15:54:22,416 INFO sqlalchemy.engine.Engine [no key 0.00116s] {}
2023-05-09 15:54:22,461 INFO sqlalchemy.engine.Engine CREATE INDEX ix_weather_df_index ON weather_df (`index`)
2023-05-09 15:54:22,462 INFO sqlalchemy.engine.Engine [no key 0.00083s] {}
2023-05-09 15:54:22,493 INFO sqlalchemy.engine.Engine COMMIT
2023-05-09 15:54:22,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-09 15:

245

In [89]:
# Top 5 destinations by with the highest temperature
df1 = pd.read_sql("SELECT cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude, MAX(weather_df.temperature) AS highest_temperature FROM weather_df INNER JOIN cities_df ON weather_df.cid = cities_df.cid GROUP BY cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude ORDER BY highest_temperature DESC LIMIT 5", engine)
df1 = pd.DataFrame(df1)
df1

2023-05-09 15:54:22,634 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:22,636 INFO sqlalchemy.engine.Engine [cached since 1.473s ago] {'table_schema': 'None', 'table_name': 'SELECT cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude, MAX(weather_df.temperature) AS highest_temperature FROM weather_df INN ... (24 characters truncated) ... ther_df.cid = cities_df.cid GROUP BY cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude ORDER BY highest_temperature DESC LIMIT 5'}
2023-05-09 15:54:22,652 INFO sqlalchemy.engine.Engine SELECT cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude, MAX(weather_df.temperature) AS highest_temperature FROM weather_df INNER JOIN cities_df ON weather_df.cid = cities_df.cid GROUP BY cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude ORDER BY highest_temperature DESC LIMI

,cid,city,latitude,longitude,highest_temperature
0,27,Nimes,43.837425,4.360069,22.58
1,0,Aigues Mortes,43.565823,4.191284,22.53
2,10,Bormes les Mimosas,43.150697,6.341928,21.58
3,5,Avignon,43.949249,4.805901,21.32
4,30,Saintes Maries de la mer,43.452277,4.428717,21.30


In [90]:
# Top 20 hotels in those areas with the highest ratings
df2 = pd.read_sql("SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df JOIN (SELECT cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude, MAX(weather_df.temperature) AS highest_temperature FROM weather_df INNER JOIN cities_df ON weather_df.cid = cities_df.cid GROUP BY cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude ORDER BY highest_temperature DESC LIMIT 5) AS top_cities ON booking_df.cid = top_cities.cid ORDER BY booking_df.rating DESC LIMIT 20", engine)
df2 = pd.DataFrame(df2)
df2

2023-05-09 15:54:22,802 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:22,803 INFO sqlalchemy.engine.Engine [cached since 1.64s ago] {'table_schema': 'None', 'table_name': 'SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df JOIN (SELECT cities_df. ... (247 characters truncated) ... ies_df.longitude ORDER BY highest_temperature DESC LIMIT 5) AS top_cities ON booking_df.cid = top_cities.cid ORDER BY booking_df.rating DESC LIMIT 20'}
2023-05-09 15:54:22,822 INFO sqlalchemy.engine.Engine SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df JOIN (SELECT cities_df.cid, cities_df.city, cities_df.latitude, cities_df.longitude, MAX(weather_df.temperature) AS highest_temperature FROM weather_df INNER JOIN cities_df ON weather_df.cid = 

,name,rating,url,cid,lat_H,lon_H
0,Roma Divine home cinéma et jardin,9.8,https://www.booking.com/hotel/fr/roma-divine-h...,27,43.835487,4.364885
1,Antichambre,9.8,https://www.booking.com/hotel/fr/antichambre.f...,27,43.835588,4.364194
2,Le Hameau de la Nicolinière,9.8,https://www.booking.com/hotel/fr/le-hameau-de-...,10,43.103481,6.327924
3,Séjour atypique et insolite sur notre péniche ...,9.7,https://www.booking.com/hotel/fr/peniche-aigue...,0,43.581709,4.208536
4,Le Repaire du Soleil,9.7,https://www.booking.com/hotel/fr/le-repaire-du...,0,43.566131,4.193800
5,"Duplex climatisé-Hauteur sous plafond 1,9m-Par...",9.6,https://www.booking.com/hotel/fr/duplex-climat...,5,43.946512,4.803512
6,HomeStay Bormes,9.6,https://www.booking.com/hotel/fr/homestay-borm...,10,43.122591,6.355040
7,Mas Fandelou,9.5,https://www.booking.com/hotel/fr/mas-fandelou....,30,43.451223,4.426668
8,Boutique Hôtel des Remparts & Spa,9.5,https://www.booking.com/hotel/fr/les-remparts-...,0,43.568036,4.190344
9,la cabane du saunier Stella - Etang du Roy,9.5,https://www.booking.com/hotel/fr/la-cabane-du-...,0,43.579302,4.212834


In [91]:
# Top 20 hotels in those areas with the highest ratings
df3 = pd.read_sql("SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df ORDER BY booking_df.rating DESC LIMIT 20", engine)
df3 = pd.DataFrame(df3)
df3

2023-05-09 15:54:22,949 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2023-05-09 15:54:22,950 INFO sqlalchemy.engine.Engine [cached since 1.786s ago] {'table_schema': 'None', 'table_name': 'SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df ORDER BY booking_df.rating DESC LIMIT 20'}
2023-05-09 15:54:22,978 INFO sqlalchemy.engine.Engine SELECT booking_df.name, booking_df.rating, booking_df.url, booking_df.cid, booking_df.lat_H, booking_df.lon_H FROM booking_df ORDER BY booking_df.rating DESC LIMIT 20
2023-05-09 15:54:22,979 INFO sqlalchemy.engine.Engine [raw sql] {}


,name,rating,url,cid,lat_H,lon_H
0,Le 6 Bis by Les Cailloux Dorés,10.0,https://www.booking.com/hotel/fr/le-6-bis-uzes...,34,44.011689,4.421285
1,"T3 Neuf Garage, clim, wifi, plages à 3min à pied!",10.0,https://www.booking.com/hotel/fr/t3-neuf-garag...,14,42.525382,3.082564
2,LE LAVOIR Classé 5***** BAYEUX Centre 9 Personnes,10.0,https://www.booking.com/hotel/fr/le-lavoir-cla...,6,49.269999,-0.706950
3,Villa Design-Piscine-BBQ-Terrasses-Centre-Ville,10.0,https://www.booking.com/hotel/fr/villa-ideale-...,15,48.070866,7.362789
4,GITE LE COQ ROUGE,9.8,https://www.booking.com/hotel/fr/gite-le-coq-r...,17,48.042576,7.305239
5,Villa Grimm,9.8,https://www.booking.com/hotel/fr/villa-grimm.f...,13,48.256709,7.450817
6,Fleur de Vigne,9.8,https://www.booking.com/hotel/fr/fleur-de-vign...,17,48.046113,7.305163
7,Apartment Le 1727,9.8,https://www.booking.com/hotel/fr/apartment-le-...,31,49.887148,2.311611
8,Le Hameau de la Nicolinière,9.8,https://www.booking.com/hotel/fr/le-hameau-de-...,10,43.103481,6.327924
9,Le Loft de La Tour Pinte,9.8,https://www.booking.com/hotel/fr/le-loft-de-la...,11,43.216262,2.286488


In [92]:
"""# we have the 20 best hotel and none of those 20 is located in Aix, that is why it is not there.
best_destination = pd.merge(df1, df2, on=['cid'])
best_destination = pd.DataFrame(best_destination)"""

"# we have the 20 best hotel and none of those 20 is located in Aix, that is why it is not there.\nbest_destination = pd.merge(df1, df2, on=['cid'])\nbest_destination = pd.DataFrame(best_destination)"

---------
### __6. Visualize our Top5 Destinations and Top 20 hotels__
---------

In [93]:
# Top 5 cities by temperature
fig1 = px.scatter_mapbox(df1,
lat='latitude', lon='longitude',
mapbox_style='carto-positron',
title='<b>Top 5 cities by temperature</b>',
hover_name = 'city',
hover_data = ['latitude', 'longitude', 'highest_temperature'],
color = 'highest_temperature',
size = 'highest_temperature', 
zoom = 6,
height = 550)

fig1.show()


In [94]:
# Top 20 hotels by ratings
# ⚠️There are actually 20 hotels, you need to zoom into each region⚠️

fig2 = px.scatter_mapbox(df2,
lat='lat_H', lon='lon_H',
mapbox_style='carto-positron',
title='<b>Top 20 Hotels in the Top 5 areas by ratings</b>',
hover_name = 'name',
hover_data = ['lat_H', 'lon_H', 'rating', 'name'],
color = 'rating',
zoom = 6,
height = 550)

fig2.show()


⚠️There are actually 20 hotels displayed in the map above, you need to zoom into each region to see them as they are really close to each other. chaging the marker size depending on the rating for instance will make the visualization more difficult⚠️

In [95]:
# Top 20 hotels from the list of our 35 cities France by ratings

fig3 = px.scatter_mapbox(df3,
lat='lat_H', lon='lon_H',
mapbox_style='carto-positron',
title='<b>Top 20 Hotels overal in France by ratings</b>',
hover_name = 'name',
hover_data = ['lat_H', 'lon_H', 'rating', 'name'],
color = 'rating',
size = 'rating',
zoom = 3.7,
height = 550)

fig3.show()


__`NOTEBOOK END`__